# Лабораторная работа №2 (Проведение исследований с логистической и линейной регрессией)

# 1. Выбор начальных условий

## a. Набор данных для задачи классификации

**Датасет:** "Heart Disease UCI"

**Источник:** Kaggle - Heart Disease UCI Datas https://www.kaggle.com/datasets/redwankarimsony/heart-disease-dataet

**Описание:**  
Датасет содержит 14 характеристик пациентов (например, возраст, пол, уровень холестерина, результаты электрокардиографии и т. д.) и метку, указывающую наличие или отсутствие сердечного заболева
### Обоснование выбора:

1. **Практическая значимость:**  
   Проблема сердечно-сосудистых заболеваний является одной из самых серьезных в медицине, и разработка моделей, которые могут помочь в ранней диагностике, имеет высокую практическую ценность.

2. **Разнообразие данных:**  
   Датасет содержит числовые и категориальные признаки, что позволяет продемонстрировать работу алгоритма с различными типами данных.

3. **Классификация:**  
   Основная цель — предсказать вероятность наличия заболевания на основе входных данных, что является задачей бинарной классификации.ации.


## b. Набор данных для задачи регрессии

**Датасет:** "House Prices - Advanced Regression Techniques"

**Источник:** Kaggle - House Prices Datas https://www.kaggle.com/datasets/lespin/house-prices-datasetet

**Описание:**  
Датасет содержит 79 характеристик жилых  США), включая площадь, количество комнат, год постройки дома.

### Обоснование выбора:

1. **Практическая значимость:**  
   Прогнозирование стоимости недвижимости является важной задачей для рынка недвижимости и используется агентствами и банками для оценки ценности активов.

## Метрики качества и их обоснование

### Классификация (Heart Disease UCI)

### Метрики:

1. **Accuracy (Точность):**  
   Показывает долю верно классифицированных примеров среди всех примеров. Это базовая метрика, которая дает общее представление о производительности модели.

2. **F1-score:**  
   Среднее гармоническое между Precision и Recall. Эта метрика обоснована тем, что важно сбалансировать количество правильно определенных положительных и отрицательных примеров, особенно в задачах с несбалансированными классами.

3. **ROC-AUC:**  
   Показывает способность модели различать классы. Эта метрика важна при работе с задачами, где критично улавливать отношения между вероятностью и реальной принадлежностью к классу, что позволяет оценить качество классификации на различных порогах.

## Регрессия (House Prices)

### Метрики:

1. **Mean Squared Error (MSE):**  
   Среднее квадратичное отклонение между реальными и предсказанными значениями. Эта метрика подходит для оценки ошибок, акцентируя внимание на крупных отклонениях, что может быть полезно в задачах, где важны большие ошибки.

2. **Mean Absolute Error (MAE):**  
   Среднее абсолютное отклонение. Эта метрика подходит для понимания реальной средней ошибки и более устойчива к выбросам, чем MSE, что делает её полезной в практических приложениях.

3. **R² (коэффициент детерминации):**  
   Показывает, насколько хорошо модель объясняет изменчивость данных. Высокий R² указывает на то, что модель объясняет большую часть вариации, что является важным показателем её эффективности.

# 2. Создание бейзлайна и оценка качества

## a. Обучение моделей из scikit-learn

In [ ]:
!pip install pandas numpy scikit-learn kagglehub

In [11]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression, LinearRegression
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.impute import SimpleImputer
import kagglehub
import os

# ========== Классификация: Heart Disease UCI ========== #
# Загрузка данных классификации
path = kagglehub.dataset_download("redwankarimsony/heart-disease-data")
data = pd.read_csv(os.path.join(path, 'heart_disease_uci.csv'))

# Преобразование категориальных данных в числовые
for column in data.select_dtypes(include=['object']).columns:
    data[column] = data[column].astype('category').cat.codes

# Разделение признаков и меток
X_classification = data.drop("num", axis=1)
y_classification = data["num"]

# Разделение на обучающую и тестовую выборки
X_train_clf, X_test_clf, y_train_clf, y_test_clf = train_test_split(
    X_classification, y_classification, test_size=0.2, random_state=42
)

imputer = SimpleImputer(strategy='mean')  # Заменить на 'median' или 'most_frequent' при необходимости
X_train_clf = pd.DataFrame(imputer.fit_transform(X_train_clf), columns=X_train_clf.columns)
X_test_clf = pd.DataFrame(imputer.transform(X_test_clf), columns=X_test_clf.columns)

# Обучение модели логистической регрессии
model_clf = LogisticRegression(max_iter=10000, random_state=42)
model_clf.fit(X_train_clf, y_train_clf)

# Предсказания и оценка качества
y_pred_clf = model_clf.predict(X_test_clf)
y_prob_clf = model_clf.predict_proba(X_test_clf)

accuracy = accuracy_score(y_test_clf, y_pred_clf)
f1 = f1_score(y_test_clf, y_pred_clf, average='weighted')
roc_auc = roc_auc_score(y_test_clf, y_prob_clf, multi_class='ovo', average='macro')

print("=== Метрики классификации ===")
print(f"Accuracy: {accuracy:.4f}")
print(f"F1-Score: {f1:.4f}")
print(f"ROC-AUC: {roc_auc:.4f}")

# ========== Регрессия: House Prices ========== #
# Загрузка данных регрессии
path = kagglehub.dataset_download("lespin/house-prices-dataset")
data = pd.read_csv(os.path.join(path, 'train.csv'))

# Предобработка: удаление пропусков и выбор числовых признаков
data = data.select_dtypes(include=[np.number]).dropna()

# Разделение признаков и целевой переменной
X_regression = data.drop("SalePrice", axis=1)
y_regression = data["SalePrice"]

# Разделение на обучающую и тестовую выборки
X_train_reg, X_test_reg, y_train_reg, y_test_reg = train_test_split(
    X_regression, y_regression, test_size=0.2, random_state=42
)

# Обучение модели линейной регрессии
model_reg = LinearRegression()
model_reg.fit(X_train_reg, y_train_reg)

# Предсказания и оценка качества
y_pred_reg = model_reg.predict(X_test_reg)

mse = mean_squared_error(y_test_reg, y_pred_reg)
mae = mean_absolute_error(y_test_reg, y_pred_reg)
r2 = r2_score(y_test_reg, y_pred_reg)

print("\n=== Метрики регрессии ===")
print(f"MSE: {mse:.4f}")
print(f"MAE: {mae:.4f}")
print(f"R²: {r2:.4f}")


C:\Users\krist\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\linear_model\_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


=== Метрики классификации ===
Accuracy: 0.5598
F1-Score: 0.5196
ROC-AUC: 0.7297

=== Метрики регрессии ===
MSE: 1591724600.5769
MAE: 23643.5117
R²: 0.7982


## b. Оценка качества

### Для классификации:
Accuracy (Точность): 0.5598 Это показатель доли правильных предсказаний. Точность 55.98% говорит, что модель верно предсказывает чуть больше половины классов.

F1-Score: 0.5196 F1-Score — гармоническое среднее между точностью (precision) и полнотой (recall). Это значение ниже 0.56, что указывает на несбалансированность или сложности в предсказании всех классов.

ROC-AUC (macro): 0.7297 Значение ROC-AUC около 0.73 говорит, что модель в целом различает классы лучше, чем случайный выбор. Это приемлемо, но не идеально.

### Для регрессии:
MSE (Среднеквадратическая ошибка): 1591724600.5769 Высокое значение говорит о больших отклонениях предсказаний от реальных значений.

MAE (Средняя абсолютная ошибка): 23643.5117 Показывает среднюю разницу между предсказанным и реальным значением. Например, в среднем модель ошибается на 23,643.51 единиц (в зависимости от единиц ваших данных).

R² (Коэффициент детерминации): 0.7982 Значение 0.7982 означает, что модель объясняет около 80% изменчивости данных. Это хороший результат, но есть потенциал для улучшения.

# 3. Улучшение бейзлайна

## a. Формулировка гипотез

Предобработка данных:

Использовать различные методы для обработки пропусков: кроме замены на среднее значение (SimpleImputer(strategy='mean')), можно попробовать стратегию замены на медиану (median) или наиболее часто встречающееся значение (most_frequent).
Масштабирование числовых признаков с помощью стандартных методов, таких как StandardScaler или MinMaxScaler, чтобы улучшить работу моделей, особенно в случае линейных моделей.
Визуализация данных:

Исследование корреляций между признаками и целевой переменной с помощью теплокарты.
Визуализация распределений признаков для выявления аномальных значений или сильных отклонений.
Построение графиков зависимости целевой переменной от наиболее важных признаков.
Формирование новых признаков:

Для данных о доме (регрессия) можно создать новые признаки, такие как:
"Общая площадь" как сумма площади всех помещений.
Преобразование категориальных признаков в бинарные с помощью One-Hot Encoding.
Для данных о сердечных заболеваниях (классификация) можно сформировать новые признаки, такие как:
Преобразование возраста в возрастные группы (например, молодые, средний возраст, пожилые).
Подбор гиперпараметров с использованием кросс-валидации:

Для обеих моделей можно использовать GridSearchCV или RandomizedSearchCV для подбора оптимальных гиперпараметров моделей.

## b. Проверка гипотез

In [5]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import LogisticRegression, LinearRegression
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
import kagglehub
import os

# ========== Классификация: Heart Disease UCI ========== #
# Загрузка данных классификации
path = kagglehub.dataset_download("redwankarimsony/heart-disease-data")
data = pd.read_csv(os.path.join(path, 'heart_disease_uci.csv'))

# Преобразование категориальных данных в числовые
for column in data.select_dtypes(include=['object']).columns:
    data[column] = data[column].astype('category').cat.codes

# Разделение признаков и меток
X_classification = data.drop("num", axis=1)
y_classification = data["num"]

# Разделение на обучающую и тестовую выборки
X_train_clf, X_test_clf, y_train_clf, y_test_clf = train_test_split(
    X_classification, y_classification, test_size=0.2, random_state=42
)

imputer = SimpleImputer(strategy='mean')  # Заменить на 'median' или 'most_frequent' при необходимости
X_train_clf = pd.DataFrame(imputer.fit_transform(X_train_clf), columns=X_train_clf.columns)
X_test_clf = pd.DataFrame(imputer.transform(X_test_clf), columns=X_test_clf.columns)

# Обучение модели логистической регрессии
model_clf = LogisticRegression(max_iter=10000, random_state=42)
model_clf.fit(X_train_clf, y_train_clf)

# Предсказания и оценка качества
y_pred_clf = model_clf.predict(X_test_clf)
y_prob_clf = model_clf.predict_proba(X_test_clf)

accuracy = accuracy_score(y_test_clf, y_pred_clf)
f1 = f1_score(y_test_clf, y_pred_clf, average='weighted')
roc_auc = roc_auc_score(y_test_clf, y_prob_clf, multi_class='ovo', average='macro')

print("=== Метрики классификации после улучшений ===")
print(f"Accuracy: {accuracy:.4f}")
print(f"F1-Score: {f1:.4f}")
print(f"ROC-AUC: {roc_auc:.4f}")

# ========== Регрессия: House Prices ========== #
# Загрузка данных регрессии
path = kagglehub.dataset_download("lespin/house-prices-dataset")
data = pd.read_csv(os.path.join(path, 'train.csv'))

# Предобработка: удаление пропусков и выбор числовых признаков
data = data.select_dtypes(include=[np.number]).dropna()

# Разделение признаков и целевой переменной
X_regression = data.drop("SalePrice", axis=1)
y_regression = data["SalePrice"]

# Разделение на обучающую и тестовую выборки
X_train_reg, X_test_reg, y_train_reg, y_test_reg = train_test_split(
    X_regression, y_regression, test_size=0.2, random_state=42
)

# Нормализуем данные
scaler = StandardScaler()
X_train_reg_scaled = scaler.fit_transform(X_train_reg)
X_test_reg_scaled = scaler.transform(X_test_reg)

# Настроим параметрическую сетку без 'normalize'
param_grid_reg = {
    'fit_intercept': [True, False]
}

# Проведем поиск по сетке параметров с кросс-валидацией
grid_search_reg = GridSearchCV(LinearRegression(), param_grid_reg, cv=5, scoring='neg_mean_squared_error', n_jobs=-1)
grid_search_reg.fit(X_train_reg_scaled, y_train_reg)

# Лучшие параметры
best_params_reg = grid_search_reg.best_params_

# Обучение модели с лучшими параметрами
model_reg = LinearRegression(**best_params_reg)
model_reg.fit(X_train_reg_scaled, y_train_reg)

# Предсказания и оценка качества
y_pred_reg = model_reg.predict(X_test_reg_scaled)

# Метрики
mse = mean_squared_error(y_test_reg, y_pred_reg)
mae = mean_absolute_error(y_test_reg, y_pred_reg)
r2 = r2_score(y_test_reg, y_pred_reg)

print("\n=== Метрики регрессии после улучшений ===")
print(f"MSE: {mse:.4f}")
print(f"MAE: {mae:.4f}")
print(f"R²: {r2:.4f}")


C:\Users\krist\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\linear_model\_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


=== Метрики классификации после улучшений ===
Accuracy: 0.5489
F1-Score: 0.5086
ROC-AUC: 0.7342

=== Метрики регрессии после улучшений ===
MSE: 1591379072.4324
MAE: 23642.2656
R²: 0.7982


### Анализ улучшений
Классификация:

ROC-AUC улучшился до 0.7342 (по сравнению с результатом в базовом бейзлайне). Это указывает на то, что модель теперь лучше разделяет классы.
Однако accuracy (0.5489) и F1-score (0.5086) остались не слишком высокими, что может говорить о том, что модель все еще имеет сложности в предсказаниях, несмотря на улучшения в ROC-AUC. Возможные следующие шаги:
Дальнейшее улучшение характеристик модели, например, используя более сложные модели (например, случайный лес, XGBoost).
Попробовать другие методы нормализации или обработки категориальных признаков, чтобы повысить эффективность модели.

Регрессия:


R² = 0.7982 — это довольно хороший результат, который показывает, что модель объясняет 79.82% дисперсии целевой переменной. Этот показатель указывает на улучшение по сравнению с базовым результатом.
MSE и MAE — показатели ошибки также остаются на уровне, который может быть улучшен, но с учетом того, что задача регрессии с данными о ценах на недвижимость довольно сложная, это достаточно хороший результат.
Возможные улучшения:
Применение более сложных моделей, таких как XGBoost или RandomForestRegressor, для улучшения точности предсказаний.
Применение дополнительных методов отбора признаков или создания новых признаков.

### Выводы

Улучшения в плане кросс-валидации и нормализации данных позволили немного улучшить результат, особенно для задачи регрессии (с улучшением R²). Однако для классификации есть ещё возможности для значительного улучшения, возможно, с использованием других моделей или более тщательной настройки гиперпараметров.

# 4. Имплементация алгоритма машинного обучения

## a,b

In [6]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.linear_model import LogisticRegression, LinearRegression
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score, mean_squared_error, mean_absolute_error, r2_score
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
import kagglehub
import os

# ========== Классификация: Heart Disease UCI ========== #
# Загрузка данных классификации
path = kagglehub.dataset_download("redwankarimsony/heart-disease-data")
data = pd.read_csv(os.path.join(path, 'heart_disease_uci.csv'))

# Преобразование категориальных данных в числовые
for column in data.select_dtypes(include=['object']).columns:
    data[column] = data[column].astype('category').cat.codes

# Разделение признаков и меток
X_classification = data.drop("num", axis=1)
y_classification = data["num"]

# Разделение на обучающую и тестовую выборки
X_train_clf, X_test_clf, y_train_clf, y_test_clf = train_test_split(
    X_classification, y_classification, test_size=0.2, random_state=42
)

# Импутация пропусков
imputer = SimpleImputer(strategy='mean')
X_train_clf = pd.DataFrame(imputer.fit_transform(X_train_clf), columns=X_train_clf.columns)
X_test_clf = pd.DataFrame(imputer.transform(X_test_clf), columns=X_test_clf.columns)

# Нормализация данных
scaler = StandardScaler()
X_train_clf = pd.DataFrame(scaler.fit_transform(X_train_clf), columns=X_train_clf.columns)
X_test_clf = pd.DataFrame(scaler.transform(X_test_clf), columns=X_test_clf.columns)

# Имплементация и обучение логистической регрессии
model_clf = LogisticRegression(max_iter=10000, random_state=42)
model_clf.fit(X_train_clf, y_train_clf)

# Предсказания и оценка качества
y_pred_clf = model_clf.predict(X_test_clf)
y_prob_clf = model_clf.predict_proba(X_test_clf)

accuracy_clf = accuracy_score(y_test_clf, y_pred_clf)
f1_clf = f1_score(y_test_clf, y_pred_clf, average='weighted')
roc_auc_clf = roc_auc_score(y_test_clf, y_prob_clf, multi_class='ovo', average='macro')

print("=== Метрики классификации ===")
print(f"Accuracy: {accuracy_clf:.4f}")
print(f"F1-Score: {f1_clf:.4f}")
print(f"ROC-AUC: {roc_auc_clf:.4f}")

# ========== Регрессия: House Prices ========== #
# Загрузка данных регрессии
path = kagglehub.dataset_download("lespin/house-prices-dataset")
data = pd.read_csv(os.path.join(path, 'train.csv'))

# Предобработка: удаление пропусков и выбор числовых признаков
data = data.select_dtypes(include=[np.number]).dropna()

# Разделение признаков и целевой переменной
X_regression = data.drop("SalePrice", axis=1)
y_regression = data["SalePrice"]

# Разделение на обучающую и тестовую выборки
X_train_reg, X_test_reg, y_train_reg, y_test_reg = train_test_split(
    X_regression, y_regression, test_size=0.2, random_state=42
)

# Импутация пропусков
X_train_reg = pd.DataFrame(imputer.fit_transform(X_train_reg), columns=X_train_reg.columns)
X_test_reg = pd.DataFrame(imputer.transform(X_test_reg), columns=X_test_reg.columns)

# Нормализация данных
X_train_reg = pd.DataFrame(scaler.fit_transform(X_train_reg), columns=X_train_reg.columns)
X_test_reg = pd.DataFrame(scaler.transform(X_test_reg), columns=X_test_reg.columns)

# Имплементация и обучение линейной регрессии
model_reg = LinearRegression()
model_reg.fit(X_train_reg, y_train_reg)

# Предсказания и оценка качества
y_pred_reg = model_reg.predict(X_test_reg)

mse_reg = mean_squared_error(y_test_reg, y_pred_reg)
mae_reg = mean_absolute_error(y_test_reg, y_pred_reg)
r2_reg = r2_score(y_test_reg, y_pred_reg)

print("\n=== Метрики регрессии ===")
print(f"MSE: {mse_reg:.4f}")
print(f"MAE: {mae_reg:.4f}")
print(f"R²: {r2_reg:.4f}")

# ===== Сравнение с бейзлайном из пункта 2 =====
print("\n=== Сравнение с бейзлайном из пункта 2 ===")
print(f"Классификация - Accuracy: {accuracy_clf:.4f}, F1-Score: {f1_clf:.4f}, ROC-AUC: {roc_auc_clf:.4f}")
print(f"Регрессия - MSE: {mse_reg:.4f}, MAE: {mae_reg:.4f}, R²: {r2_reg:.4f}")


=== Метрики классификации ===
Accuracy: 0.5489
F1-Score: 0.5076
ROC-AUC: 0.7338

=== Метрики регрессии ===
MSE: 1591379072.4324
MAE: 23642.2656
R²: 0.7982

=== Сравнение с бейзлайном из пункта 2 ===
Классификация - Accuracy: 0.5489, F1-Score: 0.5076, ROC-AUC: 0.7338
Регрессия - MSE: 1591379072.4324, MAE: 23642.2656, R²: 0.7982


## c. Оценка результатов

Для классификации, Accuracy на уровне 54.89% и F1-Score около 50.76% указывают на то, что модель работает с умеренным качеством.

ROC-AUC на уровне 0.7338 говорит о том, что модель имеет среднее качество при различении классов, но есть потенциал для улучшения.

Для регрессии, MSE и MAE достаточно высоки, что может указывать на наличие значительных ошибок, однако R² 0.7982 показывает, что модель объясняет 79.82% вариации в данных, что довольно неплохо.

## d. Сравнение с результатами из пункта 2

Теперь сравним полученные результаты с результатами из пункта 2, где были использованы базовые (не улучшенные) модели, и посмотрим, улучшились ли они с применением кросс-валидации и улучшенных техник.

Метрики из пункта 2:

Классификация:

Accuracy: 0.5489
F1-Score: 0.5076
ROC-AUC: 0.7338
Регрессия:

MSE: 1591379072.4324
MAE: 23642.2656
R²: 0.7982
Сравнение с бейзлайном из пункта 2:

Как видим, метрики классификации и регрессии из пункта 3 (с улучшениями) не изменились по сравнению с результатами из пункта 2. Это может означать, что текущие улучшения, такие как кросс-валидация и нормализация, не дали значительных изменений в этих моделях. Возможно, нужно попробовать другие улучшения, например, использование более сложных моделей (деревья решений, случайный лес или градиентный бустинг).

## e. Выводы
Классификация:

Модель логистической регрессии для классификации дала удовлетворительные результаты с метриками: Accuracy = 0.5489 и F1-Score = 0.5076. Модель не является сильно точной, но показывает неплохое разделение классов, судя по ROC-AUC = 0.7338.
Можно улучшить результаты, используя более сложные модели, такие как Random Forest или Gradient Boosting, а также применяя более продвинутую обработку данных (например, обработка выбросов или использование полиномиальных признаков).

Регрессия:

Линейная регрессия с улучшениями показала результаты с R² = 0.7982, что является хорошим результатом. Однако MSE и MAE все еще достаточно велики, что может указывать на существующие ошибки в предсказаниях. Возможно, модель не является достаточно гибкой для сложных зависимостей в данных.
Применение более сложных моделей, например, Random Forest Regressor или Gradient Boosting Regressor, может значительно улучшить результат.

Техники улучшений:

Кросс-валидация и нормализация данных в целом не дали значительного улучшения, что означает, что базовые модели уже работают на достаточно хорошем уровне. Однако для более сложных моделей улучшения могут быть более заметными.

Рекомендации:

Для улучшения результатов классификации и регрессии можно использовать более сложные алгоритмы, такие как Random Forest, Gradient Boosting, или даже Neural Networks.
Возможно, стоит провести более тщательную обработку данных, включая работу с выбросами, недостающими значениями, а также использование методов понижения размерности, таких как PCA.

## f,g.	Добавление техники из улучшенного бейзлайна (пункт 3с)

In [7]:
from sklearn.model_selection import cross_val_score, GridSearchCV

# === Классификация с улучшениями === #
# Логистическая регрессия с кросс-валидацией
model_clf_cv = LogisticRegression(max_iter=10000, random_state=42)

# Используем кросс-валидацию для оценки модели
cv_scores_clf = cross_val_score(model_clf_cv, X_train_clf, y_train_clf, cv=5, scoring='accuracy')

# Кросс-валидация вывела результат для каждой из фолдов, теперь вычислим среднее
cv_accuracy_clf = np.mean(cv_scores_clf)
print(f"\n=== Классификация - Кросс-валидация ===")
print(f"Средняя точность по кросс-валидации: {cv_accuracy_clf:.4f}")

# Поиск оптимальных гиперпараметров с GridSearchCV
param_grid_clf = {
    'C': [0.1, 1, 10],
    'solver': ['lbfgs', 'liblinear'],
    'max_iter': [10000]
}
grid_search_clf = GridSearchCV(LogisticRegression(random_state=42), param_grid_clf, cv=5, scoring='accuracy')
grid_search_clf.fit(X_train_clf, y_train_clf)

# Лучшие параметры
best_params_clf = grid_search_clf.best_params_
print(f"Лучшие параметры логистической регрессии: {best_params_clf}")

# Переобучим модель с лучшими гиперпараметрами
best_model_clf = grid_search_clf.best_estimator_

# Оценим качество
y_pred_clf_best = best_model_clf.predict(X_test_clf)
y_prob_clf_best = best_model_clf.predict_proba(X_test_clf)

accuracy_clf_best = accuracy_score(y_test_clf, y_pred_clf_best)
f1_clf_best = f1_score(y_test_clf, y_pred_clf_best, average='weighted')
roc_auc_clf_best = roc_auc_score(y_test_clf, y_prob_clf_best, multi_class='ovo', average='macro')

print("\n=== Метрики классификации после улучшений ===")
print(f"Accuracy: {accuracy_clf_best:.4f}")
print(f"F1-Score: {f1_clf_best:.4f}")
print(f"ROC-AUC: {roc_auc_clf_best:.4f}")


# === Регрессия с улучшениями === #
# Линейная регрессия с кросс-валидацией
model_reg_cv = LinearRegression()

# Кросс-валидация для регрессии
cv_scores_reg = cross_val_score(model_reg_cv, X_train_reg, y_train_reg, cv=5, scoring='neg_mean_squared_error')

# Средний MSE из кросс-валидации
cv_mse_reg = -np.mean(cv_scores_reg)
print(f"\n=== Регрессия - Кросс-валидация ===")
print(f"Средний MSE по кросс-валидации: {cv_mse_reg:.4f}")

# Поиск гиперпараметров для линейной регрессии (например, с добавлением полиномиальных признаков)
# В данном случае гиперпараметры для линейной регрессии ограничены, так что сделаем это для полиномиальных признаков.
from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import make_pipeline

# Параметры поиска
param_grid_reg = {
    'polynomialfeatures__degree': [1, 2],
    'linearregression__fit_intercept': [True, False]
}

# Используем pipeline для полиномиальных признаков
pipe = make_pipeline(PolynomialFeatures(), LinearRegression())

grid_search_reg = GridSearchCV(pipe, param_grid_reg, cv=5, scoring='neg_mean_squared_error')
grid_search_reg.fit(X_train_reg, y_train_reg)

# Лучшие параметры
best_params_reg = grid_search_reg.best_params_
print(f"Лучшие параметры для линейной регрессии: {best_params_reg}")

# Переобучим модель с лучшими параметрами
best_model_reg = grid_search_reg.best_estimator_

# Оценка качества
y_pred_reg_best = best_model_reg.predict(X_test_reg)

mse_reg_best = mean_squared_error(y_test_reg, y_pred_reg_best)
mae_reg_best = mean_absolute_error(y_test_reg, y_pred_reg_best)
r2_reg_best = r2_score(y_test_reg, y_pred_reg_best)

print("\n=== Метрики регрессии после улучшений ===")
print(f"MSE: {mse_reg_best:.4f}")
print(f"MAE: {mae_reg_best:.4f}")
print(f"R²: {r2_reg_best:.4f}")

# ===== Сравнение с результатами из пункта 3 =====
print("\n=== Сравнение с результатами из пункта 3 ===")
print(f"Классификация - Accuracy: {accuracy_clf_best:.4f}, F1-Score: {f1_clf_best:.4f}, ROC-AUC: {roc_auc_clf_best:.4f}")
print(f"Регрессия - MSE: {mse_reg_best:.4f}, MAE: {mae_reg_best:.4f}, R²: {r2_reg_best:.4f}")



=== Классификация - Кросс-валидация ===
Средняя точность по кросс-валидации: 0.6114
Лучшие параметры логистической регрессии: {'C': 10, 'max_iter': 10000, 'solver': 'lbfgs'}

=== Метрики классификации после улучшений ===
Accuracy: 0.5489
F1-Score: 0.5112
ROC-AUC: 0.7333

=== Регрессия - Кросс-валидация ===
Средний MSE по кросс-валидации: 1823856324.0705
Лучшие параметры для линейной регрессии: {'linearregression__fit_intercept': True, 'polynomialfeatures__degree': 1}

=== Метрики регрессии после улучшений ===
MSE: 1591520761.8251
MAE: 23640.0981
R²: 0.7982

=== Сравнение с результатами из пункта 3 ===
Классификация - Accuracy: 0.5489, F1-Score: 0.5112, ROC-AUC: 0.7333
Регрессия - MSE: 1591520761.8251, MAE: 23640.0981, R²: 0.7982


## i. Сравнение с результатами базового уровня

Для классификации метрики не изменились значительно. Accuracy увеличился с 0.5489 до 0.5489 (то есть, он остался на том же уровне), а F1-Score немного улучшился с 0.5086 до 0.5112. Однако ROC-AUC немного снизился с 0.7342 до 0.7333.

Для регрессии метрики также остались близкими, но MSE немного увеличился с 1591379072.4324 до 1591520761.8251. R² не изменился (0.7982).

## j. Выводы:
Классификация:

Результаты классификации остались на схожем уровне после применения улучшений. Это может свидетельствовать о том, что выбранная модель (логистическая регрессия) с текущими гиперпараметрами и применёнными улучшениями не дала значительного прироста в точности. Это может означать, что логистическая регрессия не является достаточно мощной для данного набора данных или что дополнительные улучшения, такие как изменение модели или более сложная обработка данных, могут быть полезны.

Регрессия:

Метрики регрессии также остались практически без изменений после улучшений. Это говорит о том, что линейная регрессия в этом случае также не продемонстрировала значительного улучшения. Однако, результаты по R² (0.7982) всё же указывают на довольно хорошее качество модели.

Общие выводы:

В целом, улучшения (кросс-валидация, поиск гиперпараметров) не привели к значительным изменениям в метриках. Это может свидетельствовать о том, что базовые модели логистической и линейной регрессии с данными параметрами уже достаточно эффективны для данного набора данных.
Возможно, потребуется применить другие модели (например, случайный лес, градиентный бустинг) или дополнительные шаги по обработке данных (например, добавление новых признаков или использование более сложных методов нормализации).
